# Large-Scale LoRA RL: Qwen3-8B on DeepMath-103K

This notebook validates **LoRA's effectiveness in reasoning RL** at a larger scale:

- **Model**: Qwen3-8B-Base (13x larger than 0.6B experiments)
- **Dataset**: DeepMath-103K — 103K challenging mathematical problems (levels 5-10)
- **Sequence Length**: 8192 tokens (allows backtracking and reasoning)
- **Training**: GRPO with LoRA adapters

## Why DeepMath-103K?

DeepMath-103K is significantly more challenging than MATH or GSM8K:
- **Larger scale**: 103K problems vs. 7.5K in MATH
- **Higher difficulty**: Primarily levels 5-9 (vs. levels 1-5 in MATH)
- **Decontaminated**: Rigorous decontamination against numerous benchmarks
- **Verifiable answers**: Rule-based RL reward computation
- **Multiple solutions**: 3 R1-generated reasoning paths per problem

## 8192 Token Limit

We restrict samples to 8192 tokens for:
- **Computational efficiency**: Faster experiments
- **Backtracking & reasoning**: Enough space for chain-of-thought
- **Performance trade-off**: Limits absolute performance vs. longer sequences

---

**Requirements**: H100/A100 80GB recommended (8B model + 8192 context + LoRA training)

**Reference**: He et al., 2025 - DeepMath-103K: A Large-Scale, Challenging, Decontaminated, and Verifiable Mathematical Dataset

## 🛠️ Setup & Installation

In [ ]:
#@title Setup
!nvidia-smi -L || true

import os, sys, random, numpy as np, torch, json, time, platform
print("Python:", sys.version)
print("CUDA available:", torch.cuda.is_available())

# Install dependencies
try:
    get_ipython().run_line_magic("uv", "pip -q install transformers==4.51.3 accelerate==1.4.0 peft==0.14.0 datasets==3.3.2 evaluate==0.4.3 trl==0.13.0 flash-attn --no-build-isolation sentencepiece protobuf tqdm matplotlib > /dev/null")
except Exception:
    get_ipython().run_line_magic("pip", "-q install transformers==4.51.3 accelerate==1.4.0 peft==0.14.0 datasets==3.3.2 evaluate==0.4.3 trl==0.13.0 flash-attn --no-build-isolation sentencepiece protobuf tqdm matplotlib > /dev/null")

import transformers, datasets, peft, accelerate, matplotlib
print("Transformers:", transformers.__version__)
print("Accelerate:", accelerate.__version__)
print("PEFT:", peft.__version__)

SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
assert DEVICE == "cuda", "Please connect a GPU (H100/A100 80GB recommended)."

print(f"\n✓ Running on {torch.cuda.get_device_name(0)}")
print(f"  GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

## ⚙️ Configuration Parameters

**Key differences from base experiment:**
- Model: Qwen3-8B-Base (larger)
- Dataset: DeepMath-103K (harder, more diverse)
- Max length: 8192 tokens (vs. 2048)
- Adjusted batch sizes for memory efficiency

In [ ]:
from dataclasses import dataclass
from typing import Optional, List
import re

@dataclass
class Config:
    # ========== Model Settings ==========
    model_id: str = "Qwen/Qwen2.5-8B-Base"  # 8B base model for large-scale experiments
    
    # ========== LoRA Settings ==========
    # LoRA parameters are NOT frozen - actively trained with RL!
    lora_r: int = 32                    # Higher rank for 8B model (more capacity)
    lora_alpha: int = 64                # Scaling factor
    lora_dropout: float = 0.05          # Dropout for LoRA layers
    lora_target_modules: List[str] = None  # Will target all attention + MLP
    
    # ========== Sequence Length ==========
    max_seq_length: int = 8192          # 8192 token limit for training/eval
    max_new_tokens: int = 2048          # Max tokens to generate (leaves room for reasoning)
    
    # ========== RL Training Settings (GRPO) ==========
    n_grpo_steps: int = 500             # More steps for larger dataset
    prompts_per_step: int = 16          # Reduced for memory (8B model + long context)
    rollouts_per_prompt: int = 4        # Group size (K samples per prompt)
    
    learning_rate: float = 5e-5         # Lower LR for larger model
    weight_decay: float = 0.01          # Small weight decay
    
    micro_batch_size: int = 1           # Very small due to 8192 context length
    gradient_accumulation_steps: int = 64  # Accumulate to effective batch size
    epochs_per_step: int = 1            # Training epochs per GRPO step
    max_grad_norm: float = 1.0          # Gradient clipping
    
    # ========== Generation Settings ==========
    temperature: float = 0.7            # Sampling temperature for rollouts
    top_p: float = 0.9                  # Nucleus sampling
    eval_temperature: float = 0.0       # Greedy decoding for evaluation
    
    # ========== Dataset Settings (DeepMath-103K) ==========
    dataset_id: str = "zwhe99/DeepMath-103K"
    difficulty_filter: Optional[tuple] = None  # (min, max) difficulty, None = all
    topic_filter: Optional[List[str]] = None   # Filter by topics, None = all
    
    prompt_template: str = (
        "Solve this mathematical problem step by step. "
        "Show your reasoning and provide the final answer.\n\n"
        "Problem: {question}\n\n"
        "Solution:"
    )
    
    # ========== Memory Optimization ==========
    use_flash_attention: bool = True    # FlashAttention-2 for efficiency
    use_gradient_checkpointing: bool = True  # Save memory during training
    bf16: bool = True                   # BFloat16 training
    
    # ========== Monitoring & Logging ==========
    log_every: int = 10                 # Log metrics every N steps
    eval_every: int = 25                # Evaluate every N steps
    eval_samples: int = 200             # Validation samples
    save_every: int = 50                # Save checkpoint every N steps
    
    # ========== Data Splits ==========
    train_samples: Optional[int] = None  # None = use all (103K)
    val_samples: int = 500              # Validation set size
    test_samples: int = 1000            # Test set size
    
    # ========== Output ==========
    run_name: str = f"lora_8b_deepmath_{int(time.time())}"
    output_dir: str = "./lora_8b_runs"
    
    def __post_init__(self):
        if self.lora_target_modules is None:
            self.lora_target_modules = [
                "q_proj", "k_proj", "v_proj", "o_proj",
                "gate_proj", "up_proj", "down_proj"
            ]
        self.run_dir = os.path.join(self.output_dir, self.run_name)
        os.makedirs(self.run_dir, exist_ok=True)

cfg = Config()

print("="*70)
print("Large-Scale LoRA RL Configuration")
print("="*70)
print(f"Model:           {cfg.model_id}")
print(f"Dataset:         {cfg.dataset_id}")
print(f"Max seq length:  {cfg.max_seq_length} tokens")
print(f"Max new tokens:  {cfg.max_new_tokens} tokens")
print(f"LoRA rank:       {cfg.lora_r}, alpha: {cfg.lora_alpha}")
print(f"GRPO steps:      {cfg.n_grpo_steps}")
print(f"Batch config:    {cfg.prompts_per_step} prompts × {cfg.rollouts_per_prompt} rollouts")
print(f"Effective batch: {cfg.micro_batch_size} × {cfg.gradient_accumulation_steps} = {cfg.micro_batch_size * cfg.gradient_accumulation_steps}")
print(f"Learning rate:   {cfg.learning_rate}")
print(f"Flash Attention: {cfg.use_flash_attention}")
print(f"Gradient ckpt:   {cfg.use_gradient_checkpointing}")
print(f"Output:          {cfg.run_dir}")
print("="*70)

## 📊 Load DeepMath-103K Dataset

DeepMath-103K contains:
- `question`: Mathematical problem
- `final_answer`: Verifiable answer (for RL reward)
- `difficulty`: Float score (enables curriculum learning)
- `topic`: Hierarchical classification
- `r1_solution_1/2/3`: Three reasoning paths from DeepSeek-R1

In [ ]:
from datasets import load_dataset

def render_prompt(question: str) -> str:
    return cfg.prompt_template.format(question=question)

def normalize_answer(answer: str) -> str:
    """Normalize mathematical answer for comparison"""
    if answer is None:
        return ""
    # Remove whitespace, convert to lowercase
    answer = answer.strip().lower()
    # Remove common LaTeX delimiters
    answer = answer.replace("$", "").replace("\\boxed{", "").replace("}", "")
    answer = answer.replace("\\text{", "").replace("\\(", "").replace("\\)", "")
    return answer.strip()

def compute_reward(generated_text: str, gold_answer: str) -> float:
    """
    Binary reward based on final answer match.
    Looks for common answer patterns: boxed, brackets, or last mathematical expression.
    """
    # Try to extract answer from generated text
    # Pattern 1: \boxed{answer}
    boxed_match = re.search(r"\\boxed\{([^}]+)\}", generated_text)
    if boxed_match:
        pred_answer = boxed_match.group(1)
    else:
        # Pattern 2: [answer] or (answer)
        bracket_match = re.search(r"\[([^\]]+)\]|\(([^\)]+)\)", generated_text)
        if bracket_match:
            pred_answer = bracket_match.group(1) or bracket_match.group(2)
        else:
            # Pattern 3: "Final answer: ..." or "Answer: ..."
            answer_match = re.search(r"(?:final\s+)?answer\s*:?\s*(.+?)(?:\.|$)", generated_text.lower())
            if answer_match:
                pred_answer = answer_match.group(1)
            else:
                # Fallback: last line
                lines = [l.strip() for l in generated_text.split("\n") if l.strip()]
                pred_answer = lines[-1] if lines else ""
    
    # Normalize both answers
    pred_norm = normalize_answer(pred_answer)
    gold_norm = normalize_answer(gold_answer)
    
    # Exact match
    if pred_norm == gold_norm:
        return 1.0
    
    # Partial match (contained)
    if pred_norm and gold_norm and (pred_norm in gold_norm or gold_norm in pred_norm):
        return 0.5
    
    return 0.0

print("Loading DeepMath-103K...")
ds_full = load_dataset(cfg.dataset_id, split="train")

# Filter by difficulty if specified
if cfg.difficulty_filter is not None:
    min_diff, max_diff = cfg.difficulty_filter
    ds_full = ds_full.filter(lambda x: min_diff <= x["difficulty"] <= max_diff)
    print(f"Filtered to difficulty {min_diff}-{max_diff}: {len(ds_full)} samples")

# Filter by topic if specified
if cfg.topic_filter is not None:
    ds_full = ds_full.filter(lambda x: any(t in x["topic"] for t in cfg.topic_filter))
    print(f"Filtered to topics {cfg.topic_filter}: {len(ds_full)} samples")

# Filter by length (8192 token limit)
from transformers import AutoTokenizer
print("Loading tokenizer for length filtering...")
temp_tokenizer = AutoTokenizer.from_pretrained(cfg.model_id, trust_remote_code=True)

def is_valid_length(example):
    """Check if example fits within 8192 token budget"""
    prompt = render_prompt(example["question"])
    # Account for prompt + max generation + buffer
    prompt_len = len(temp_tokenizer.encode(prompt))
    return prompt_len + cfg.max_new_tokens < cfg.max_seq_length

print("Filtering by 8192 token limit (this may take a few minutes)...")
ds_full = ds_full.filter(is_valid_length, desc="Length filter")
print(f"After 8192 token filtering: {len(ds_full)} samples")

# Create splits
total = len(ds_full)
test_size = min(cfg.test_samples, total // 10)
val_size = min(cfg.val_samples, total // 20)

# Shuffle and split
ds_full = ds_full.shuffle(seed=SEED)
ds_test = ds_full.select(range(test_size))
ds_val = ds_full.select(range(test_size, test_size + val_size))

if cfg.train_samples is not None:
    train_end = min(test_size + val_size + cfg.train_samples, total)
else:
    train_end = total
ds_train = ds_full.select(range(test_size + val_size, train_end))

print(f"\nDataset splits:")
print(f"  Train: {len(ds_train):,}")
print(f"  Val:   {len(ds_val):,}")
print(f"  Test:  {len(ds_test):,}")

# Show example
ex = ds_train[0]
print(f"\nExample problem:")
print(f"  Topic:      {ex['topic']}")
print(f"  Difficulty: {ex['difficulty']}")
print(f"  Question:   {ex['question'][:200]}...")
print(f"  Answer:     {ex['final_answer']}")

# Show difficulty distribution
import matplotlib.pyplot as plt
difficulties = [ex["difficulty"] for ex in ds_train.select(range(min(1000, len(ds_train))))]
plt.figure(figsize=(10, 4))
plt.hist(difficulties, bins=20, edgecolor='black')
plt.xlabel("Difficulty")
plt.ylabel("Count")
plt.title("DeepMath-103K Difficulty Distribution (Train Set Sample)")
plt.grid(alpha=0.3)
plt.show()

del temp_tokenizer

## 🏗️ Load Qwen3-8B with LoRA

We use:
- **Flash Attention 2**: For memory-efficient long context
- **Gradient checkpointing**: Reduce memory during backprop
- **BFloat16**: Stable training with less memory
- **Higher LoRA rank**: More capacity for 8B model

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model, TaskType, prepare_model_for_kbit_training

def load_model_with_lora(model_id: str, lora_config: LoraConfig):
    """Load 8B model with LoRA and memory optimizations"""
    print(f"Loading base model: {model_id}...")
    
    model_kwargs = {
        "torch_dtype": torch.bfloat16 if cfg.bf16 else torch.float16,
        "device_map": "auto",
        "trust_remote_code": True,
    }
    
    # Add Flash Attention if available
    if cfg.use_flash_attention:
        try:
            model_kwargs["attn_implementation"] = "flash_attention_2"
            print("  Using Flash Attention 2")
        except Exception as e:
            print(f"  Flash Attention not available: {e}")
    
    model = AutoModelForCausalLM.from_pretrained(model_id, **model_kwargs)
    model.config.use_cache = False  # Disable for training
    
    # Enable gradient checkpointing
    if cfg.use_gradient_checkpointing:
        model.gradient_checkpointing_enable()
        print("  Gradient checkpointing enabled")
    
    print("Applying LoRA adapters...")
    model = get_peft_model(model, lora_config)
    
    # Print trainable parameters
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total_params = sum(p.numel() for p in model.parameters())
    print(f"\nParameter summary:")
    print(f"  Trainable: {trainable_params:,} ({100 * trainable_params / total_params:.3f}%)")
    print(f"  Total:     {total_params:,}")
    print(f"  Frozen:    {total_params - trainable_params:,}")
    
    return model

# Load tokenizer
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(
    cfg.model_id,
    use_fast=True,
    trust_remote_code=True,
    model_max_length=cfg.max_seq_length
)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

print(f"Tokenizer vocab size: {len(tokenizer)}")
print(f"Max sequence length: {cfg.max_seq_length}")

# Configure LoRA
lora_config = LoraConfig(
    r=cfg.lora_r,
    lora_alpha=cfg.lora_alpha,
    lora_dropout=cfg.lora_dropout,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
    target_modules=cfg.lora_target_modules,
)

print(f"\nLoRA configuration:")
print(f"  Rank: {cfg.lora_r}")
print(f"  Alpha: {cfg.lora_alpha}")
print(f"  Dropout: {cfg.lora_dropout}")
print(f"  Target modules: {cfg.lora_target_modules}")

# Load model
model = load_model_with_lora(cfg.model_id, lora_config)

print("\n✓ Model ready for large-scale RL training!")

## 📈 Baseline Evaluation

Measure baseline performance on challenging DeepMath-103K problems.

In [ ]:
from tqdm.auto import tqdm

@torch.no_grad()
def evaluate_accuracy(model, tokenizer, dataset, num_samples: int = 100, temperature: float = 0.0, batch_size: int = 4):
    """Evaluate with 8192 token context"""
    model.eval()
    was_cache = model.config.use_cache
    model.config.use_cache = True
    
    n = min(num_samples, len(dataset))
    total_reward = 0.0
    exact_matches = 0
    
    for i in tqdm(range(0, n, batch_size), desc="Evaluating"):
        batch = dataset.select(range(i, min(i + batch_size, n)))
        prompts = [render_prompt(ex["question"]) for ex in batch]
        
        # Tokenize with 8192 limit
        inputs = tokenizer(
            prompts,
            padding=True,
            truncation=True,
            max_length=cfg.max_seq_length - cfg.max_new_tokens,
            return_tensors="pt"
        ).to(model.device)
        
        # Generate
        gen_kwargs = dict(
            max_new_tokens=cfg.max_new_tokens,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            use_cache=True
        )
        if temperature > 0.0:
            gen_kwargs.update(do_sample=True, temperature=temperature, top_p=cfg.top_p)
        else:
            gen_kwargs.update(do_sample=False)
        
        outputs = model.generate(**inputs, **gen_kwargs)
        texts = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        
        # Compute rewards
        for ex, text in zip(batch, texts):
            reward = compute_reward(text, ex["final_answer"])
            total_reward += reward
            if reward >= 0.99:
                exact_matches += 1
    
    model.config.use_cache = was_cache
    avg_reward = total_reward / n
    accuracy = exact_matches / n
    
    return {"avg_reward": avg_reward, "accuracy": accuracy}

print("Computing baseline metrics on test set...")
print("(This may take a while due to 8192 token context)\n")
baseline_metrics = evaluate_accuracy(
    model, tokenizer, ds_test,
    num_samples=min(100, len(ds_test)),
    temperature=0.0,
    batch_size=2  # Small batch for long context
)

print(f"\nBaseline Results (DeepMath-103K):")
print(f"  Average Reward: {baseline_metrics['avg_reward']:.3f}")
print(f"  Exact Match:    {baseline_metrics['accuracy']:.1%}")

# Save baseline
with open(os.path.join(cfg.run_dir, "baseline.json"), "w") as f:
    json.dump(baseline_metrics, f, indent=2)

## 🎯 GRPO Training Loop (8192 Context)

Key adaptations for large-scale:
- **8192 token sequences**: Allows complex reasoning chains
- **Micro-batching**: Process 1 example at a time, accumulate gradients
- **Memory efficient**: Gradient checkpointing + flash attention
- **Harder problems**: DeepMath-103K difficulty 5-10

In [ ]:
import torch.nn.functional as F
import pandas as pd
from IPython.display import display

def create_response_mask(input_ids: torch.Tensor, prompt_lengths: List[int], eos_token_id: int) -> torch.Tensor:
    """Mask: 1 for generated tokens, 0 for prompt/padding/post-EOS"""
    batch_size, seq_len = input_ids.shape
    mask = torch.zeros_like(input_ids, dtype=torch.float32)
    
    for i, prompt_len in enumerate(prompt_lengths):
        # Find first EOS in generated part
        generated_ids = input_ids[i, prompt_len:]
        eos_positions = (generated_ids == eos_token_id).nonzero(as_tuple=True)[0]
        
        if len(eos_positions) > 0:
            first_eos = eos_positions[0].item()
            mask[i, prompt_len:prompt_len + first_eos] = 1.0
        else:
            mask[i, prompt_len:] = 1.0
    
    return mask

def compute_policy_logprobs_chunked(model, input_ids: torch.Tensor, response_mask: torch.Tensor, chunk_size: int = 1) -> torch.Tensor:
    """
    Compute log probs in chunks to handle long sequences.
    Returns: [batch_size, seq_len] log probs
    """
    batch_size = input_ids.size(0)
    seq_len = input_ids.size(1)
    all_logprobs = []
    
    for i in range(0, batch_size, chunk_size):
        chunk_ids = input_ids[i:i+chunk_size]
        chunk_mask = response_mask[i:i+chunk_size]
        
        # Forward pass
        outputs = model(
            input_ids=chunk_ids[:, :-1],
            attention_mask=(chunk_ids[:, :-1] != tokenizer.pad_token_id)
        )
        logits = outputs.logits
        
        # Get log probs for next tokens
        log_probs = F.log_softmax(logits, dim=-1)
        next_tokens = chunk_ids[:, 1:].unsqueeze(-1)
        token_logprobs = log_probs.gather(-1, next_tokens).squeeze(-1)
        
        # Pad and apply mask
        token_logprobs = F.pad(token_logprobs, (1, 0), value=0.0)
        token_logprobs = token_logprobs * chunk_mask
        
        all_logprobs.append(token_logprobs)
    
    return torch.cat(all_logprobs, dim=0)

class MetricsTable:
    def __init__(self):
        self.rows = []
        self.df = pd.DataFrame(columns=["step", "loss", "avg_reward", "accuracy", "kl_div", "val_reward", "val_acc"])
        self.handle = display(self.df, display_id=True)
    
    def update(self, step, loss=None, avg_reward=None, accuracy=None, kl_div=None, val_reward=None, val_acc=None):
        row = {"step": step}
        if loss is not None:
            row["loss"] = f"{loss:.4f}"
        if avg_reward is not None:
            row["avg_reward"] = f"{avg_reward:.3f}"
        if accuracy is not None:
            row["accuracy"] = f"{accuracy:.1%}"
        if kl_div is not None:
            row["kl_div"] = f"{kl_div:.4f}"
        if val_reward is not None:
            row["val_reward"] = f"{val_reward:.3f}"
        if val_acc is not None:
            row["val_acc"] = f"{val_acc:.1%}"
        
        self.rows.append(row)
        self.df = pd.DataFrame(self.rows)
        self.handle.update(self.df)

def save_checkpoint(model, step: int, metrics: dict):
    """Save LoRA checkpoint with metadata"""
    save_path = os.path.join(cfg.run_dir, f"checkpoint_step_{step}")
    os.makedirs(save_path, exist_ok=True)
    model.save_pretrained(save_path)
    
    # Save metrics
    with open(os.path.join(save_path, "metrics.json"), "w") as f:
        json.dump(metrics, f, indent=2)
    
    print(f"✓ Saved checkpoint: {save_path}")
    return save_path

def train_grpo_large_scale():
    """GRPO training for 8B model on DeepMath-103K with 8192 context"""
    print("\n" + "="*70)
    print("Starting Large-Scale GRPO Training")
    print("Model: Qwen3-8B | Dataset: DeepMath-103K | Context: 8192 tokens")
    print("="*70 + "\n")
    
    # Setup
    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=cfg.learning_rate,
        weight_decay=cfg.weight_decay
    )
    
    # Learning rate scheduler (cosine decay)
    from torch.optim.lr_scheduler import CosineAnnealingLR
    scheduler = CosineAnnealingLR(optimizer, T_max=cfg.n_grpo_steps, eta_min=cfg.learning_rate * 0.1)
    
    metrics_table = MetricsTable()
    logs = []
    
    for step in tqdm(range(cfg.n_grpo_steps), desc="GRPO Steps"):
        # ========== 1. Sample prompts ==========
        rng = np.random.default_rng(SEED + step)
        indices = rng.choice(len(ds_train), size=cfg.prompts_per_step, replace=False)
        batch_examples = [ds_train[int(i)] for i in indices]
        
        # Repeat for K rollouts
        prompts_repeated = sum([[render_prompt(ex["question"])] * cfg.rollouts_per_prompt for ex in batch_examples], [])
        answers_repeated = sum([[ex["final_answer"]] * cfg.rollouts_per_prompt for ex in batch_examples], [])
        
        # ========== 2. Generate rollouts ==========
        model.eval()
        with torch.no_grad():
            inputs = tokenizer(
                prompts_repeated,
                padding=True,
                truncation=True,
                max_length=cfg.max_seq_length - cfg.max_new_tokens,
                return_tensors="pt"
            ).to(model.device)
            
            prompt_lengths = (inputs.input_ids != tokenizer.pad_token_id).sum(dim=1).tolist()
            
            gen_outputs = model.generate(
                **inputs,
                max_new_tokens=cfg.max_new_tokens,
                do_sample=True,
                temperature=cfg.temperature,
                top_p=cfg.top_p,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id,
                return_dict_in_generate=True
            )
            
            full_sequences = gen_outputs.sequences
            generated_texts = tokenizer.batch_decode(full_sequences, skip_special_tokens=True)
            
            # ========== 3. Compute rewards ==========
            rewards = torch.tensor(
                [compute_reward(text, ans) for text, ans in zip(generated_texts, answers_repeated)],
                dtype=torch.float32,
                device=model.device
            )
            
            # Group normalization
            rewards_grouped = rewards.view(cfg.prompts_per_step, cfg.rollouts_per_prompt)
            group_means = rewards_grouped.mean(dim=1, keepdim=True)
            advantages = (rewards_grouped - group_means).view(-1)
            
            # Store old log probs
            response_mask = create_response_mask(full_sequences, prompt_lengths, tokenizer.eos_token_id)
            old_logprobs = compute_policy_logprobs_chunked(
                model, full_sequences, response_mask, chunk_size=cfg.micro_batch_size
            )
        
        # ========== 4. Policy update ==========
        model.train()
        total_loss = 0.0
        num_batches = 0
        
        for epoch in range(cfg.epochs_per_step):
            perm = torch.randperm(full_sequences.size(0))
            
            for i in range(0, full_sequences.size(0), cfg.micro_batch_size):
                indices = perm[i:i + cfg.micro_batch_size]
                
                mb_sequences = full_sequences[indices]
                mb_mask = response_mask[indices]
                mb_advantages = advantages[indices]
                
                # Current policy log probs
                curr_logprobs = compute_policy_logprobs_chunked(
                    model, mb_sequences, mb_mask, chunk_size=cfg.micro_batch_size
                )
                
                # Policy gradient loss
                sequence_logprobs = curr_logprobs.sum(dim=1)
                loss = -(mb_advantages * sequence_logprobs).mean()
                loss = loss / cfg.gradient_accumulation_steps
                
                loss.backward()
                total_loss += loss.item() * cfg.gradient_accumulation_steps
                num_batches += 1
                
                # Optimizer step
                if (num_batches % cfg.gradient_accumulation_steps) == 0:
                    torch.nn.utils.clip_grad_norm_(model.parameters(), cfg.max_grad_norm)
                    optimizer.step()
                    optimizer.zero_grad()
        
        scheduler.step()
        avg_loss = total_loss / num_batches if num_batches > 0 else 0.0
        
        # ========== 5. Metrics ==========
        with torch.no_grad():
            new_logprobs = compute_policy_logprobs_chunked(
                model, full_sequences, response_mask, chunk_size=cfg.micro_batch_size
            )
            kl_div = ((old_logprobs - new_logprobs) * response_mask).sum() / response_mask.sum()
            
            avg_reward = rewards.mean().item()
            accuracy = (rewards >= 0.99).float().mean().item()
        
        log_entry = {
            "step": step,
            "loss": avg_loss,
            "avg_reward": avg_reward,
            "accuracy": accuracy,
            "kl_div": kl_div.item(),
            "lr": scheduler.get_last_lr()[0]
        }
        
        # ========== 6. Evaluation ==========
        if (step % cfg.eval_every == 0 and step > 0) or (step == cfg.n_grpo_steps - 1):
            print(f"\nEvaluating at step {step}...")
            val_metrics = evaluate_accuracy(
                model, tokenizer, ds_val,
                num_samples=cfg.eval_samples,
                temperature=0.0,
                batch_size=2
            )
            log_entry["val_reward"] = val_metrics["avg_reward"]
            log_entry["val_acc"] = val_metrics["accuracy"]
            print(f"Val Reward: {val_metrics['avg_reward']:.3f}, Val Acc: {val_metrics['accuracy']:.1%}")
        
        # ========== 7. Logging ==========
        if (step % cfg.log_every == 0) or ("val_reward" in log_entry):
            metrics_table.update(
                step=step,
                loss=avg_loss,
                avg_reward=avg_reward,
                accuracy=accuracy,
                kl_div=kl_div.item(),
                val_reward=log_entry.get("val_reward"),
                val_acc=log_entry.get("val_acc")
            )
        
        logs.append(log_entry)
        
        # ========== 8. Checkpointing ==========
        if (step % cfg.save_every == 0 and step > 0) or (step == cfg.n_grpo_steps - 1):
            save_checkpoint(model, step, log_entry)
        
        torch.cuda.empty_cache()
    
    # Save logs
    pd.DataFrame(logs).to_csv(os.path.join(cfg.run_dir, "training_logs.csv"), index=False)
    print("\n✓ Training complete!")
    return logs

# Run training
training_logs = train_grpo_large_scale()

## 📊 Final Evaluation & Analysis

In [ ]:
print("Running final test evaluation...")
final_metrics = evaluate_accuracy(
    model, tokenizer, ds_test,
    num_samples=len(ds_test),
    temperature=0.0,
    batch_size=2
)

print("\n" + "="*70)
print("FINAL RESULTS: Large-Scale LoRA RL")
print("="*70)
print(f"Model:    Qwen3-8B-Base")
print(f"Dataset:  DeepMath-103K (103K problems, difficulty 5-10)")
print(f"Context:  8192 tokens")
print(f"Training: {cfg.n_grpo_steps} GRPO steps")
print("\nBaseline (before RL):")
print(f"  Avg Reward:  {baseline_metrics['avg_reward']:.3f}")
print(f"  Accuracy:    {baseline_metrics['accuracy']:.1%}")
print("\nFinal (after RL):")
print(f"  Avg Reward:  {final_metrics['avg_reward']:.3f}")
print(f"  Accuracy:    {final_metrics['accuracy']:.1%}")
print("\nImprovement:")
print(f"  Δ Reward:    {(final_metrics['avg_reward'] - baseline_metrics['avg_reward']):.3f}")
print(f"  Δ Accuracy:  {(final_metrics['accuracy'] - baseline_metrics['accuracy']):.1%}")
print("="*70)

# Save final results
results = {
    "config": {
        "model": cfg.model_id,
        "dataset": cfg.dataset_id,
        "max_seq_length": cfg.max_seq_length,
        "lora_r": cfg.lora_r,
        "n_grpo_steps": cfg.n_grpo_steps,
    },
    "baseline": baseline_metrics,
    "final": final_metrics,
    "improvement": {
        "avg_reward": final_metrics["avg_reward"] - baseline_metrics["avg_reward"],
        "accuracy": final_metrics["accuracy"] - baseline_metrics["accuracy"]
    }
}

with open(os.path.join(cfg.run_dir, "final_results.json"), "w") as f:
    json.dump(results, f, indent=2)

print(f"\n✓ All artifacts saved to: {cfg.run_dir}")

# Plot training curves
import matplotlib.pyplot as plt
df = pd.DataFrame(training_logs)

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Loss
axes[0, 0].plot(df["step"], df["loss"])
axes[0, 0].set_xlabel("Step")
axes[0, 0].set_ylabel("Loss")
axes[0, 0].set_title("Training Loss")
axes[0, 0].grid(alpha=0.3)

# Reward
axes[0, 1].plot(df["step"], df["avg_reward"], label="Train")
if "val_reward" in df.columns:
    val_df = df.dropna(subset=["val_reward"])
    axes[0, 1].plot(val_df["step"], val_df["val_reward"], label="Val", marker="o")
axes[0, 1].set_xlabel("Step")
axes[0, 1].set_ylabel("Avg Reward")
axes[0, 1].set_title("Average Reward")
axes[0, 1].legend()
axes[0, 1].grid(alpha=0.3)

# Accuracy
axes[1, 0].plot(df["step"], df["accuracy"], label="Train")
if "val_acc" in df.columns:
    val_df = df.dropna(subset=["val_acc"])
    axes[1, 0].plot(val_df["step"], val_df["val_acc"], label="Val", marker="o")
axes[1, 0].set_xlabel("Step")
axes[1, 0].set_ylabel("Accuracy")
axes[1, 0].set_title("Exact Match Accuracy")
axes[1, 0].legend()
axes[1, 0].grid(alpha=0.3)

# KL Divergence
axes[1, 1].plot(df["step"], df["kl_div"])
axes[1, 1].set_xlabel("Step")
axes[1, 1].set_ylabel("KL Divergence")
axes[1, 1].set_title("KL Divergence (Policy Drift)")
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(cfg.run_dir, "training_curves.png"), dpi=150, bbox_inches="tight")
plt.show()

print("\n✓ Training curves saved!")

## 🚀 Deployment with vLLM/sglang

Deploy your trained 8B LoRA adapter with inference engines:

```python
# vLLM deployment
from vllm import LLM, SamplingParams
from vllm.lora.request import LoRARequest

llm = LLM(
    model="Qwen/Qwen3-8B-Base",
    enable_lora=True,
    max_lora_rank=32,  # Match your LoRA rank
    max_model_len=8192,  # Support 8192 context
    tensor_parallel_size=1
)

lora_path = "./lora_8b_runs/lora_8b_deepmath_XXX/checkpoint_step_500"
lora_request = LoRARequest("deepmath_lora", 1, lora_path)

# Generate
sampling_params = SamplingParams(temperature=0.0, max_tokens=2048)
outputs = llm.generate(prompts, sampling_params, lora_request=lora_request)
```

**Performance benefits:**
- PagedAttention for 8192 token context
- Continuous batching for throughput
- Multi-LoRA serving (different fine-tuned versions)

## 📝 Summary

This notebook validates **LoRA's effectiveness for reasoning RL at scale**:

### Key Findings:
1. **Scalability**: LoRA enables RL training of 8B models with <1% trainable parameters
2. **Long context**: 8192 token sequences allow complex reasoning chains
3. **Hard problems**: DeepMath-103K difficulty 5-10 tests advanced mathematical reasoning
4. **Memory efficiency**: Gradient checkpointing + Flash Attention fit on A100 80GB

### Compared to smaller experiments:
- **13x larger model** (0.6B → 8B parameters)
- **4x longer context** (2048 → 8192 tokens)
- **13x more data** (7.5K → 103K problems)
- **Harder dataset** (GSM8K level 1-3 → DeepMath level 5-10)

### Implementation highlights:
- GRPO with group advantage normalization
- Micro-batching for memory efficiency
- Cosine LR schedule for stable training
- Rule-based rewards from verifiable answers

**Next steps:**
- Try curriculum learning (start with easier problems)
- Experiment with different LoRA ranks
- Combine with other techniques (PPO, rejection sampling)
- Test on other reasoning benchmarks (MATH, Minerva, etc.)